In [ ]:
# stdlib
import os

# syft absolute
import syft as sy
from syft import test_settings
from syft.util.test_helpers.worker_helpers import (
    build_and_launch_worker_pool_from_docker_str,
)
from syft.util.test_helpers.worker_helpers import (
    launch_worker_pool_from_docker_tag_and_registry,
)

In [ ]:
# stdlib
environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
high_port = os.environ.get("CLUSTER_HTTP_PORT_HIGH", "9081")
low_port = os.environ.get("CLUSTER_HTTP_PORT_LOW", "9083")
print(environment, high_port, low_port)

# Launch server & login

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port=low_port,
)

server_high = sy.orchestra.launch(
    name="bigquery-high",
    server_side_type="high",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port=high_port,
)

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"
low_client = sy.login(
    url=f"http://localhost:{low_port}", email=ADMIN_EMAIL, password=ADMIN_PW
)
high_client = sy.login(
    url=f"http://localhost:{high_port}", email=ADMIN_EMAIL, password=ADMIN_PW
)

# Setup High First

- If using an external registery, we want to get this from the test_settings.
- We build the docker image over the base docker image in Syft
- We give a tag called worker-bigquery to our custom pool image

In [ ]:
external_registry = test_settings.get("external_registry", default="docker.io")

base_worker_image = high_client.images.get_all()[0]

worker_dockerfile = f"""
FROM {str(base_worker_image.image_identifier)}

RUN uv pip install db-dtypes google-cloud-bigquery 

""".strip()

docker_tag = str(base_worker_image.image_identifier).replace(
    "backend", "worker-bigquery"
)

In [ ]:
worker_pool_name = "bigquery-pool"
custom_pool_pod_annotations = {"bigquery-custom-pool": "Pod annotation for bigquery"}
custom_pool_pod_labels = {"bigquery-custom-pool": "Pod_label_for_bigquery"}

In [ ]:
build_and_launch_worker_pool_from_docker_str(
    environment=environment,
    client=high_client,
    worker_pool_name=worker_pool_name,
    custom_pool_pod_annotations=custom_pool_pod_annotations,
    custom_pool_pod_labels=custom_pool_pod_labels,
    worker_dockerfile=worker_dockerfile,
    external_registry=external_registry,
    docker_tag=docker_tag,
    scale_to=2,
)

In [ ]:
assert len(high_client.worker_pools.get_all()) == 2

In [ ]:
high_client.settings.allow_guest_signup(enable=False)

# Setup Low

In [ ]:
launch_result = launch_worker_pool_from_docker_tag_and_registry(
    environment=environment,
    client=low_client,
    worker_pool_name=worker_pool_name,
    custom_pool_pod_annotations=custom_pool_pod_annotations,
    custom_pool_pod_labels=custom_pool_pod_labels,
    docker_tag=docker_tag,
    external_registry=external_registry,
    scale_to=1,
)

In [ ]:
assert len(low_client.worker_pools.get_all()) == 2

# Register a DS only on the low side

In [ ]:
low_client.register(
    email="data_scientist@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="John Doe",
)

In [ ]:
low_client.settings.allow_guest_signup(enable=False)

In [ ]:
assert (
    len(low_client.api.services.user.get_all()) == 3
), "Only DS and 2 Admin should be at low side"
assert (
    len(high_client.api.services.user.get_all()) == 2
), "Only 2 Admin should be at high side"

# Close

In [ ]:
if environment != "remote":
    server_high.land()
    server_low.land()